# <center> Импорт библоиотек <center>

*NOTE*: не забудьте составить список библиотек, необходимых для реализации ваших решений, и представить их в файле `requirements.txt`.

In [502]:
import pickle
import random
import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score

# <center> Чтение данных <center>

Тренировчные данные представлены в виде файла в формате JSON. В котором верхенеуровневый ключ -- это номер образца (бакетрии). Каждый словарь по бактериям содержит название штамма и результаты масс-спектрометрического анализа, которые представлены следующими полями: масса к заряду (m/z), время (time), интенсивность пика (Intens.), разрешение (Res.), площадь пика (Area), относительная интенсивность (Rel. Intens.), ширина на полувысоте (FWHM=full width at half-maximum intensity). Каждый штамм будет представлен несколькими бактериями.

In [503]:
with open('input/train.json', 'rb') as fp:
    train_d = json.load(fp)
train_df = pd.DataFrame(json.loads(train_d)).T

In [504]:
train_df.shape

(256, 10)

In [505]:
## Посмотрим сколько классов данных и сколько примеров на каждый класс
train_df.groupby('strain').agg({'strain': "count"})

,strain
strain,
Acinetobacter baumani_121 skin,9
Acinetobacter baumani_126,9
Acinetobacter baumani_352 blood,10
Acinetobacter baumani_377 blood,11
Acinetobacter baumani_503 blood,11
Acinetobacter baumani_63,9
Acinetobacter baumani_64,11
Acinetobacter baumani_73,10
Acinetobacter baumani_74,11


Так как тренировочный набор данных не очень большой, дополним его симмулированными данными. Для этого составим дополнительные примеры из существующих. Зададим seed для воспроизводимости результатов.

In [506]:
# для начала орпеделим количестов пиков в масс-спектрах
train_df.loc[:,'n_peak'] = train_df['m/z'].apply(len)

In [507]:
def get_dif(x):
    return max(x) - min(x)

In [508]:
# посмотрим на сколько по количеству строк (пиков) различаются таблицы внутри каждого штамма
peaks_dif = train_df.groupby(['strain']).agg({'n_peak': get_dif})

In [510]:
# будем использовать среднюю разности для варьирования числа пиков в сгенерированных данных
mean_dif = int(peaks_dif.n_peak.mean()) 

In [511]:
FEATURES = ['m/z', 'time', 'Intens.', 'SN', 'Res.', 'Area', 'Rel. Intens.',
            'FWHM', 'Bk. Peak']

In [512]:
np.random.seed(142)
train_gen = pd.DataFrame()
for strain in train_df.strain.unique():
    tmp = train_df[train_df.strain == strain]
    n_samples = tmp.shape[0]
    s = np.random.randint(max(tmp.n_peak) - mean_dif, max(tmp.n_peak), 6)
    strain_df = pd.DataFrame()
    
    for i,sample in enumerate(tmp[FEATURES].values):
        tmp_i = pd.DataFrame(list(sample)).T
        tmp_i.columns = FEATURES
        strain_df = pd.concat([strain_df, tmp_i])
    for i in range(0,6):
        df_i = pd.DataFrame()
        idx = []
        for n in range(0,s[i]):
            if isinstance(strain_df.loc[n], pd.Series):
                continue
            else:
                df_i = pd.concat([df_i, strain_df.loc[n].sample(n=1)])
        df_i[['id']] = i
        df_i.loc[:, 'strain'] = tmp.strain.unique()[0]
        df_i.loc[:, 'n_peak'] = n
        train_gen = pd.concat([train_gen, df_i])        

In [513]:
train_gen.shape

(12349, 12)

In [514]:
train_gen = train_gen.groupby(['strain', 'id']).agg(list).reset_index().drop(['id'], axis = 1)
train_gen.loc[:,'n_peak'] = train_gen.n_peak.apply(min)

C:\Users\micapic\AppData\Local\Temp\ipykernel_7648\3242894118.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train_gen.loc[:,'n_peak'] = train_gen.n_peak.apply(min)


Совместим оригинальных датасет и сгенерированный.

In [515]:
train_df = pd.concat([train_df, train_gen[train_df.columns]])

In [516]:
train_df.shape

(412, 11)

# <center> Обработка данных и генерация признаков<center>

Пример таблицы с данными по масс-спектру для одного образца

In [517]:
example = train_df[FEATURES]\
         .loc[(train_df.strain == 'Staphilococcus aureus_6 1006')].iloc[0]
example = pd.DataFrame(list(example)).T
example.columns = FEATURES

In [518]:
example.head()

,m/z,time,Intens.,SN,Res.,Area,Rel. Intens.,FWHM,Bk. Peak
0,2141.595860,46248.308185,1849.671417,5.023842,449.793188,13621.062834,0.084247,4.761290,0.0
1,2154.140758,46381.493984,1299.384644,3.529223,458.732871,9421.672911,0.059183,4.695850,0.0
2,2180.436755,46659.416149,1203.332001,3.317415,443.973316,9161.078120,0.054808,4.911189,0.0
3,2297.759326,47879.408034,1426.807159,3.967446,620.345018,9085.341876,0.064987,3.704002,0.0
4,2764.201381,52445.360112,1586.883118,4.470495,629.209920,12546.815080,0.072278,4.393131,0.0


In [520]:
## select uncorrelated features
FEATURES = ['m/z', 'Rel. Intens.', 'Res.', 'FWHM']

Посмотрим на масс-спектры для нескольих примеров

Создаим спектры, которые удобно сравнитвать между собой в виде векторов в фиксированом диапазоне, и уберем необходимость использовать переменную "m/z".

In [524]:
# орпеделим диапазон парамтеров
min(train_df['m/z'].apply(min)), max(train_df['m/z'].apply(max))

(2040.804289776974, 17241.93492389016)

In [525]:
def create_speactr(mz, intens):
    spec = []
    for i in range(200, 1750):
        if i in mz:
            spec.append(intens[mz.index(i)])
        else:
            spec.append(0)
    return spec

In [526]:
def prepocess_data(data):
    data['mz'] = data['m/z'].apply(lambda x: [int(x_i // 10) for x_i in x])
    data['intens'] = data.apply(lambda d: create_speactr(d['mz'], d['Rel. Intens.']),
                           axis = 1)
    return data

In [527]:
train_df = prepocess_data(train_df)

Перемешаем данные, чтобы классы не шли по порядку

In [528]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

# <center> Обучение моделей <center>

Разобьем выборку на валидационную и тренировочную так, чтобы хотя бы один пример из каждого класса присутствавал
в валидаицонной выборке. После разбиения уберем два класса из тренировочный выборки, чтобы можно было определить их 
как 'new'.

_Note_:  рекомендуем рассмотреть разные вариатны разбиения на тренировочную и валидационную выборку для определения оптиматльных парпамтеров. 

In [529]:
np.random.seed(142)
val_ds = pd.DataFrame()
train_ds = pd.DataFrame()
for strain in train_df.strain.unique():
    n = np.random.randint(1,4)
    ids = random.sample(list(train_df.loc[train_df.strain == strain].index), n)
    val_ds = pd.concat([val_ds, train_df.loc[ids]])
    train_ids = list(set(train_df.loc[train_df.strain == strain].index) - set(ids))
    train_ds = pd.concat([train_ds, train_df.loc[train_ids]])


In [530]:
val_ds.shape, train_ds.shape

((54, 13), (358, 13))

In [531]:
random.seed(111)
val_strain = random.sample(sorted(list(val_ds.strain.unique())), 2)
val_strain

['Acinetobacter baumani_64', 'Pseudomonas fluorescence_965']

In [532]:
train_ds = train_ds[~train_ds.strain.isin(val_strain)]

In [533]:
val_ds.loc[:,'target'] = val_ds.strain
val_ds.loc[val_ds.strain.isin(val_strain),'target'] = 'new'

In [534]:
val_ds = val_ds.sample(frac=1)
train_ds = train_ds.sample(frac=1)

In [535]:
def make_x(df, col):
    X = []
    for i in df.index:
        row = df.loc[i,col]
        X.append(row)
    return np.array(X)

## Нейросетевой классификатор

Закодируем штаммы:

In [536]:
from sklearn.preprocessing import LabelEncoder
train_ds['strain_encoded'] = LabelEncoder().fit_transform(train_ds['strain'])
train_df['strain_encoded'] = LabelEncoder().fit_transform(train_df['strain'])

strains = train_ds[['strain_encoded', 'strain']].drop_duplicates()
decoder_sample = strains.set_index('strain_encoded').to_dict()['strain']

strains = train_df[['strain_encoded', 'strain']].drop_duplicates()
decoder_full = strains.set_index('strain_encoded').to_dict()['strain']
pickle.dump(decoder_full, open(f'models/decoder.pkl', 'wb'))

Инициализируем модель и обучим ее

In [537]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.losses import SparseCategoricalCrossentropy
from tensorflow.python.keras.regularizers import l2

model = Sequential([
    Dense(64, input_shape=[len(train_ds['intens'].head(1).item())], activation='relu', kernel_regularizer=l2(0.0001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),
    Dense(len(decoder_sample), activation='linear')
  ])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [538]:
X = make_x(train_ds, col='intens')
y = np.array(train_ds['strain_encoded'])

In [539]:
model.fit(X, y, epochs=50)

Epoch 1/50
11/11 [==============================] - 0s 6ms/step - loss: 3.1696 - accuracy: 0.1159
Epoch 2/50
11/11 [==============================] - 0s 7ms/step - loss: 3.0929 - accuracy: 0.2287
Epoch 3/50
11/11 [==============================] - 0s 8ms/step - loss: 2.9843 - accuracy: 0.2226
Epoch 4/50
11/11 [==============================] - 0s 8ms/step - loss: 2.8320 - accuracy: 0.2348
Epoch 5/50
11/11 [==============================] - 0s 6ms/step - loss: 2.6338 - accuracy: 0.2470
Epoch 6/50
11/11 [==============================] - 0s 6ms/step - loss: 2.4223 - accuracy: 0.2622
Epoch 7/50
11/11 [==============================] - 0s 7ms/step - loss: 2.2270 - accuracy: 0.3110
Epoch 8/50
11/11 [==============================] - 0s 6ms/step - loss: 2.0469 - accuracy: 0.4177
Epoch 9/50
11/11 [==============================] - 0s 6ms/step - loss: 1.8914 - accuracy: 0.5122
Epoch 10/50
11/11 [==============================] - 0s 4ms/step - loss: 1.7460 - accuracy: 0.6311
Epoch 11/50
11/11 [

### Проверяем качество модели на валидационной выборке

In [540]:
X_pred = make_x(val_ds, col='intens')

In [541]:
res = model.predict(X_pred)
class_name = []
for res_i in res:
    if max(res_i) > 1.0:
        class_name.append(decoder_sample[res_i.argmax()])
    else:
        class_name.append('new')

2/2 [==============================] - 0s 3ms/step


In [542]:
val_ds.loc[:,'class_name'] = class_name
print(f"Accuracy: {1 - len(val_ds['target'].compare(val_ds['class_name'])) / len(val_ds)}")

Accuracy: 0.9074074074074074


In [543]:
print(f"F1-score: {f1_score(val_ds.class_name, val_ds.target, average='macro')}")

F1-score: 0.8855238095238094


Заново обучим модель на всей тренировочной выборке, так как мы исключали классы для валидации

In [544]:
X = make_x(train_df, col='intens')
y = np.array(train_df['strain_encoded'])

model = Sequential([
    Dense(64, input_shape=[len(train_df['intens'].head(1).item())], activation='relu', kernel_regularizer=l2(0.0001)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.0001)),
    Dense(len(decoder_full), activation='linear')
  ])

model.compile(optimizer='adam',
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X, y, epochs=50)

Epoch 1/50
13/13 [==============================] - 1s 5ms/step - loss: 3.2480 - accuracy: 0.0728
Epoch 2/50
13/13 [==============================] - 0s 5ms/step - loss: 3.1552 - accuracy: 0.2743
Epoch 3/50
13/13 [==============================] - 0s 5ms/step - loss: 3.0029 - accuracy: 0.3301
Epoch 4/50
13/13 [==============================] - 0s 5ms/step - loss: 2.7805 - accuracy: 0.3447
Epoch 5/50
13/13 [==============================] - 0s 5ms/step - loss: 2.5145 - accuracy: 0.3835
Epoch 6/50
13/13 [==============================] - 0s 5ms/step - loss: 2.2367 - accuracy: 0.4490
Epoch 7/50
13/13 [==============================] - 0s 5ms/step - loss: 1.9905 - accuracy: 0.4490
Epoch 8/50
13/13 [==============================] - 0s 4ms/step - loss: 1.7936 - accuracy: 0.5558
Epoch 9/50
13/13 [==============================] - 0s 5ms/step - loss: 1.6307 - accuracy: 0.6990
Epoch 10/50
13/13 [==============================] - 0s 4ms/step - loss: 1.4906 - accuracy: 0.7233
Epoch 11/50
13/13 [

In [545]:
model.save("models/nn")

INFO:tensorflow:Assets written to: models/nn\assets
